[PyCon US 2024 - YouTube](https://www.youtube.com/playlist?list=PL2Uw4_HvXqvYhjub9bw4uDAmNtprgAvlJ)

In [ ]:
!pip install -Uq pytube

In [ ]:
import sqlite3  # sqlite3 라이브러리 임포트 (Python 내장 라이브러리)
from pytube import Playlist  # pytube 라이브러리에서 Playlist 클래스 임포트
from pytube import YouTube  # YouTube 동영상 제목 가져오기 위해 추가
from youtube_transcript_api import YouTubeTranscriptApi  # YouTubeTranscriptApi 라이브러리 임포트
import pandas as pd  # 판다스 라이브러리 임포트
import logging  # logging 라이브러리 임포트

# 로그 설정
logging.basicConfig(
    level=logging.INFO,  # 로그 레벨 설정
    format='%(asctime)s - %(levelname)s - %(message)s',  # 로그 형식 설정
    handlers=[
        logging.FileHandler("subtitle_collection.log", encoding='utf-8'),  # 파일에 로그 저장
        logging.StreamHandler()  # 콘솔에 로그 출력
    ]
)

# 사용할 YouTube 재생목록 URL (사용자가 제공한 URL)
playlist_url = "https://www.youtube.com/watch?v=OH3C5a2uAWA&list=PL2Uw4_HvXqvYhjub9bw4uDAmNtprgAvlJ"

# 자막 형식을 HH:MM:SS로 변환하는 함수
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# 1단계: YouTube 재생목록에서 모든 URL 가져오기
playlist = Playlist(playlist_url)  # Playlist 객체 생성
video_urls = [video_url for video_url in playlist.video_urls]  # 재생목록의 모든 동영상 URL 수집

# 2단계: SQLite 데이터베이스 연결
conn = sqlite3.connect('youtube_playlist.db')  # youtube_playlist.db 파일로 SQLite 데이터베이스 연결 (파일이 없으면 새로 생성)
cursor = conn.cursor()  # 데이터베이스 작업을 위한 커서 객체 생성

# 3단계: URL, 제목, 자막을 저장할 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS videos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,  -- 고유 ID를 위한 자동 증가 정수형 필드
    url TEXT NOT NULL UNIQUE,  -- URL을 저장할 텍스트 필드 (고유해야 함)
    title TEXT,  -- 동영상 제목을 저장할 텍스트 필드
    subtitles TEXT  -- 자막을 저장할 텍스트 필드
)
''')

# 4단계: 테이블에 동영상 URL, 제목과 자막 삽입 또는 자막이 없는 경우만 수집
for url in video_urls:
    video_id = url.split("v=")[-1].split("&")[0]  # URL에서 video_id 추출
    
    # 자막이 이미 있는지 확인
    cursor.execute('SELECT subtitles FROM videos WHERE url = ?', (url,))
    result = cursor.fetchone()
    
    if result and result[0] is not None:  # 자막이 이미 있는 경우 스킵
        logging.info(f"동영상 {video_id}의 자막이 이미 존재합니다. 스킵합니다.")
        continue
    
    try:
        # 동영상 제목 가져오기
        yt = YouTube(url)
        title = yt.title
        
        # 자막 가져오기 (기본 언어)
        transcript = YouTubeTranscriptApi.get_transcript(video_id)  # 기본 언어의 자막 가져오기
        
        # 자막을 포맷팅하여 하나의 문자열로 합치기
        subtitles = "\n".join([f"[{format_time(entry['start'])}] {entry['text']}" for entry in transcript])
        logging.info(f"동영상 {video_id}의 제목과 자막을 성공적으로 가져왔습니다.")
    except Exception as e:
        logging.error(f"동영상 {video_id}의 자막을 가져오는 중 오류가 발생했습니다: {str(e)}")
        title = yt.title if 'yt' in locals() else "제목 없음"  # 제목을 가져오지 못했을 경우 기본 값 설정
        subtitles = "자막 없음"  # 자막을 가져오지 못했을 경우 기본 값 설정
    
    # URL, 제목, 자막을 데이터베이스에 삽입 또는 업데이트
    cursor.execute('INSERT INTO videos (url, title, subtitles) VALUES (?, ?, ?) ON CONFLICT(url) DO UPDATE SET title = ?, subtitles = ?',
                   (url, title, subtitles, title, subtitles))

# 변경 사항 저장 및 데이터베이스 연결 종료
conn.commit()  # 데이터베이스에 변경 사항 커밋 (저장)
logging.info("동영상 URL, 제목, 자막이 'youtube_playlist.db' 데이터베이스에 저장되었습니다.")

# 5단계: 데이터베이스에서 저장된 URL, 제목과 자막 확인
logging.info("저장된 동영상 URL, 제목, 자막을 출력합니다:")
cursor.execute('SELECT url, title, subtitles FROM videos')
rows = cursor.fetchall()

for row in rows:
    logging.info(f"URL: {row[0]}")
    logging.info(f"제목: {row[1]}")
    # logging.info(f"자막:\n{row[2]}\n{'-' * 80}\n")

# 6단계: 데이터베이스에서 전체 데이터를 읽어와 판다스 데이터프레임으로 변환 및 출력
df = pd.read_sql_query("SELECT * FROM videos", conn)  # SQLite 쿼리 결과를 데이터프레임으로 변환
print(df)

# 데이터베이스 연결 종료
conn.close()


In [ ]:
df

In [ ]:
df.iloc[-5]["subtitles"][:300]